In [2]:
%pip install ifcopenshell
%pip install lark numpy ## some package missing in the ifcopenshell

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
file_path = 'AC20-FZK-Haus.ifc'

In [4]:
from utils import get_materials, get_material_layer_sets, get_material_layer_set_usages
import ifcopenshell
model = ifcopenshell.open(file_path)
materials = get_materials(model)
materialSets = get_material_layer_sets(model)
materialSetUsages = get_material_layer_set_usages(model)

print(materials)
print(materialSets)
print(materialSetUsages)

[#14521=IfcMaterial('Holz',$,$), #15046=IfcMaterial('Leichtbeton 102890359',$,$), #20378=IfcMaterial('Radial Gradient Fill 1515460218',$,$), #34513=IfcMaterial('Stahlbeton 65690',$,$), #35176=IfcMaterial('Solid',$,$), #59294=IfcMaterial('Solid 397409098',$,$)]
[#15069=IfcMaterialLayerSet((#15067),'Leichtbeton 102890359 0.24',$), #21971=IfcMaterialLayerSet((#21970),'Leichtbeton 102890359 0.3',$), #34527=IfcMaterialLayerSet((#34526),'Stahlbeton 65690 0.2',$), #59316=IfcMaterialLayerSet((#59315),'Solid 397409098 0.2',$)]
[#15072=IfcMaterialLayerSetUsage(#15069,.AXIS2.,.NEGATIVE.,0.,$), #17044=IfcMaterialLayerSetUsage(#15069,.AXIS2.,.POSITIVE.,-0.,$), #18469=IfcMaterialLayerSetUsage(#15069,.AXIS2.,.POSITIVE.,-0.,$), #18702=IfcMaterialLayerSetUsage(#15069,.AXIS2.,.POSITIVE.,-0.,$), #20602=IfcMaterialLayerSetUsage(#15069,.AXIS2.,.NEGATIVE.,0.,$), #21974=IfcMaterialLayerSetUsage(#21971,.AXIS2.,.NEGATIVE.,0.,$), #27425=IfcMaterialLayerSetUsage(#21971,.AXIS2.,.NEGATIVE.,0.,$), #31474=IfcMateria

In [5]:
elements = model.by_type('IfcElement')
print(elements)

[#14502=IfcStair('38a9vdh9bF5Qg28GWyHhlr',#12,'Wendeltreppe',$,$,#502,#14498,'79A67A01-C95B-4209-86-9A-74983B65305C',$), #15042=IfcWallStandardCase('2XPyKWY018sA1ygZKgQPtU',#12,'Wand-Int-ERDG-4',$,$,#14983,#15037,'BC6F0F70-6195-495E-A2-FC-239713029DB1',$), #17040=IfcWallStandardCase('3PfS__Y_DBAfq5naM6zD2Z',#12,'Wand-Int-ERDG-2',$,$,#16993,#17035,'40F78310-9E7E-4793-BC-9E-DB102FD72B73',$), #17106=IfcOpeningElement('0LM8GvGe$G3dlW4mZ4aA9R',#12,'Innentuer-1',$,$,#17061,#17102,'15588439-428F-D00E-7BE0-1308C490A25B',$), #17468=IfcDoor('1Oms875aH3Wg$9l65H2ZGw',#12,'Innentuer-1',$,$,#17302,#17464,'E605AACB-6C4E-458D-93-46-5D9D8C7C21D3',2.01,0.885,$,$,$), #18465=IfcWallStandardCase('2ptk1k7qn8_Qk22vjh$0DE',#12,'Wand-Int-ERDG-1',$,$,#18418,#18460,'8C826359-B394-49D6-BD-D5-49D50F806B56',$), #18698=IfcWallStandardCase('3jjW3rL656ex34Gws22EfM',#12,'Wand-Int-ERDG-3',$,$,#18651,#18693,'2197C9B9-DEF1-444F-B8-1E-1DB092762AB7',$), #18768=IfcOpeningElement('253rr8ORNAORFOx911Dvez',#12,'Innentuer-2',$,$

In [11]:
associations = model.by_type('IfcRelAssociatesMaterial')
for association in associations:
    related_materials = association.RelatingMaterial
    related_objects = association.RelatedObjects
    print(related_materials)
    print(related_objects)

#14521=IfcMaterial('Holz',$,$)
(#14502=IfcStair('38a9vdh9bF5Qg28GWyHhlr',#12,'Wendeltreppe',$,$,#502,#14498,'79A67A01-C95B-4209-86-9A-74983B65305C',$),)
#15072=IfcMaterialLayerSetUsage(#15069,.AXIS2.,.NEGATIVE.,0.,$)
(#15042=IfcWallStandardCase('2XPyKWY018sA1ygZKgQPtU',#12,'Wand-Int-ERDG-4',$,$,#14983,#15037,'BC6F0F70-6195-495E-A2-FC-239713029DB1',$),)
#17044=IfcMaterialLayerSetUsage(#15069,.AXIS2.,.POSITIVE.,-0.,$)
(#17040=IfcWallStandardCase('3PfS__Y_DBAfq5naM6zD2Z',#12,'Wand-Int-ERDG-2',$,$,#16993,#17035,'40F78310-9E7E-4793-BC-9E-DB102FD72B73',$),)
#17475=IfcMaterialList((#14521))
(#17468=IfcDoor('1Oms875aH3Wg$9l65H2ZGw',#12,'Innentuer-1',$,$,#17302,#17464,'E605AACB-6C4E-458D-93-46-5D9D8C7C21D3',2.01,0.885,$,$,$), #19199=IfcDoor('0pGAjlJMP3ifYPATVF5xAR',#12,'Innentuer-2',$,$,#19029,#19195,'CBFAA406-7368-4C3F-96-65-F3ED0FC60C2F',2.01,0.885,$,$,$), #19504=IfcDoor('2qiPPF3FrF8OIqfrKiSUqm',#12,'Innentuer-3',$,$,#19486,#19500,'12291272-8C4D-42C2-A4-E5-68086A355C33',2.01,0.885,$,$,$), #23

In [8]:
## TODO: move to utils
def extract_material_usage(model):
    ifc_file = model
    material_usage_info = {}

    # Iterate over all materials
    for material in ifc_file.by_type("IfcMaterial"):
        used_in_elements = set()
        # Find associations to elements
        for association in ifc_file.by_type("IfcRelAssociatesMaterial"):
            if association.RelatingMaterial == material or \
               (association.RelatingMaterial.is_a("IfcMaterialLayerSetUsage") and association.RelatingMaterial.ForLayerSet == material):
                for relatedObject in association.RelatedObjects:
                    element_type = relatedObject.is_a()
                    element_id = relatedObject.GlobalId
                    # Determine the spatial structure (location)
                    if relatedObject.ContainedInStructure:
                        for rel_contained in relatedObject.ContainedInStructure:
                            structure = rel_contained.RelatingStructure
                            location = structure.LongName or structure.Name
                            if structure.is_a("IfcSpace"):  # If you want more specific space info
                                location += f" in {structure.LongName or structure.Name}"
                            used_in_elements.add((element_type, element_id, location))
        
        material_usage_info[material.Name] = used_in_elements

    return material_usage_info

material_usage_info = extract_material_usage_with_location(model)
print(material_usage_info)

{'Holz': {('IfcStair', '38a9vdh9bF5Qg28GWyHhlr', 'ACID00000001-0000-0000-0000-000000000000')}, 'Leichtbeton 102890359': set(), 'Radial Gradient Fill 1515460218': {('IfcBeam', '3tCgZT92j6fw8fXgwCL3Jm', 'ACID00000001-0000-0000-0000-000000000000')}, 'Stahlbeton 65690': set(), 'Solid': {('IfcMember', '3qFc3Wrr9BBPJyb2wB$4Ra', 'ACID00000002-0000-0000-0000-000000000000'), ('IfcRailing', '0o5vgCKyTBzO5$QJcI2YDP', 'ACID00000002-0000-0000-0000-000000000000'), ('IfcMember', '09OmJpUcj4WvNp2dXHFMWj', 'ACID00000002-0000-0000-0000-000000000000'), ('IfcMember', '1ulHlNTuj798Os3k9phhVC', 'ACID00000002-0000-0000-0000-000000000000'), ('IfcMember', '1NBDw_i_jD$u7IjvwmbPmx', 'ACID00000002-0000-0000-0000-000000000000'), ('IfcMember', '2in0QvfYXEawl1f3MP1bq2', 'ACID00000002-0000-0000-0000-000000000000'), ('IfcBeam', '20bTaetQDApP5w8egFxj13', 'ACID00000002-0000-0000-0000-000000000000'), ('IfcMember', '0nsN_jkIrBQBuGFMg_zMEg', 'ACID00000002-0000-0000-0000-000000000000'), ('IfcMember', '23pmL_2aH88OVqWp_5Lfkf